# Install libraries

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip install keybert

In [4]:
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
import numpy as np
from typing import List, Tuple
import re
from collections import defaultdict
from keybert import KeyBERT
from tqdm.notebook import tqdm
import time
import pickle
from datetime import datetime
import os
import pandas as pd
import string

# Load dataset SemEval

In [5]:
# Load dataset
ds = load_dataset("SemEvalWorkshop/sem_eval_2010_task_8")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.23k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/673k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/231k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2717 [00:00<?, ? examples/s]

## See some samples

In [6]:
# Take first 10 samples for testing
test_samples = list(ds['train'].select(range(10)))

# Display first sample structure
print("Sample data structure:")
print(test_samples[0])

# Display all 10 sentences and their entities
print("\nSamples with their entities:")
for i, sample in enumerate(test_samples):
    entities = re.findall(r'<e[12]>(.*?)</e[12]>', sample['sentence'])
    print(f"\n{i+1}. Sentence: {sample['sentence']}")
    print(f"   Entities: {entities}")

Sample data structure:
{'sentence': 'The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.', 'relation': 3}

Samples with their entities:

1. Sentence: The system as described above has its greatest application in an arrayed <e1>configuration</e1> of antenna <e2>elements</e2>.
   Entities: ['configuration', 'elements']

2. Sentence: The <e1>child</e1> was carefully wrapped and bound into the <e2>cradle</e2> by means of a cord.
   Entities: ['child', 'cradle']

3. Sentence: The <e1>author</e1> of a keygen uses a <e2>disassembler</e2> to look at the raw assembly code.
   Entities: ['author', 'disassembler']

4. Sentence: A misty <e1>ridge</e1> uprises from the <e2>surge</e2>.
   Entities: ['ridge', 'surge']

5. Sentence: The <e1>student</e1> <e2>association</e2> is the voice of the undergraduate student population of the State University of New York at Buffalo.
   Entities: ['student', 'association']

6. Sentence: Th

### check n_gram range

In [7]:
import pandas as pd
df = pd.DataFrame(list(ds['train']))
df

,sentence,relation
0,The system as described above has its greatest...,3
1,The <e1>child</e1> was carefully wrapped and b...,18
2,The <e1>author</e1> of a keygen uses a <e2>dis...,11
3,A misty <e1>ridge</e1> uprises from the <e2>su...,18
4,The <e1>student</e1> <e2>association</e2> is t...,12
...,...,...
7995,When the <e1>notice</e1> is sent by <e2>fax</e...,18
7996,The <e1>herbicide</e1> is derived from a natur...,8
7997,"To test this, we placed a kitchen <e1>match</e...",6
7998,The farmers and city officials in the region h...,18


In [8]:
entity = []
for sample in df['sentence']:
    entities = re.findall(r'<e[12]>(.*?)</e[12]>', sample)
    entity.append(entities)

In [9]:
df['true_entities'] = entity

In [10]:
df

,sentence,relation,true_entities
0,The system as described above has its greatest...,3,"[configuration, elements]"
1,The <e1>child</e1> was carefully wrapped and b...,18,"[child, cradle]"
2,The <e1>author</e1> of a keygen uses a <e2>dis...,11,"[author, disassembler]"
3,A misty <e1>ridge</e1> uprises from the <e2>su...,18,"[ridge, surge]"
4,The <e1>student</e1> <e2>association</e2> is t...,12,"[student, association]"
...,...,...,...
7995,When the <e1>notice</e1> is sent by <e2>fax</e...,18,"[notice, fax]"
7996,The <e1>herbicide</e1> is derived from a natur...,8,"[herbicide, antibiotic]"
7997,"To test this, we placed a kitchen <e1>match</e...",6,"[match, jar]"
7998,The farmers and city officials in the region h...,18,"[farmers, market]"


In [11]:
# Find the maximum number of words in an entity
max_words = 0
for entities in df['true_entities']:
    for entity in entities:
        words = entity.split()
        max_words = max(max_words, len(words))

print(f"The maximum number of words in an entity is: {max_words}")

The maximum number of words in an entity is: 3


In [12]:
  # prompt: print entities which have 3 words

# Assuming df and its 'true_entities' column are defined as in the previous code.

three_word_entities = []
for entities in df['true_entities']:
    for entity in entities:
        words = entity.split()
        if len(words) == 3:
            three_word_entities.append(entity)

print("Entities with three words:")
for entity in three_word_entities:
    print(entity)


Entities with three words:
outer-surface protein A
safe deposit box
stainless steel tray
ice cream maker
wire mesh basket
credit card bill
right party members
quake recovery projects
arterial blood pressure
editor cum scientist
rising unemployment rate
mobile software applications
space shuttle orbiters
male reproduction biology
road construction projects
nuclear aircraft carriers
stem cell research
online shopping malls
central nervous system
social security number
turn signal indicator
candy bar wrapper
positive health light
fenced back yard
open central space


#### So each entity at most has 3 words, it tells us that it might better to have longer entities

#### check nr_candidate

In [13]:
lenghts = []
for entities in df['true_entities']:
    length = len(entities)
    lenghts.append(length)

In [14]:
df['length'] = lenghts
df

,sentence,relation,true_entities,length
0,The system as described above has its greatest...,3,"[configuration, elements]",2
1,The <e1>child</e1> was carefully wrapped and b...,18,"[child, cradle]",2
2,The <e1>author</e1> of a keygen uses a <e2>dis...,11,"[author, disassembler]",2
3,A misty <e1>ridge</e1> uprises from the <e2>su...,18,"[ridge, surge]",2
4,The <e1>student</e1> <e2>association</e2> is t...,12,"[student, association]",2
...,...,...,...,...
7995,When the <e1>notice</e1> is sent by <e2>fax</e...,18,"[notice, fax]",2
7996,The <e1>herbicide</e1> is derived from a natur...,8,"[herbicide, antibiotic]",2
7997,"To test this, we placed a kitchen <e1>match</e...",6,"[match, jar]",2
7998,The farmers and city officials in the region h...,18,"[farmers, market]",2


In [15]:
max(lenghts)

2

In [16]:
min(lenghts)

2

#### It shows us that just 2 keywords extracted from the text, so nr_candidate should be at least 2.

# Load model & define functions

### Add preprocessing

In [7]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove mentions (@user)
    text = re.sub(r'@\w+', '', text)

    # Remove hashtags (#hashtag)
    text = re.sub(r'#\w+', '', text)

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [8]:
def clean_sentence(text: str) -> str:
    """Remove XML tags from text"""
    return re.sub(r'</?e[12]>', '', text).strip()


def extract_keyphrases(ngram, nr_cand, diversity, text: str, model: KeyBERT, top_n: int = 2) -> List[str]:
    """Extract keyphrases using KeyBERT"""
    keyphrases = model.extract_keywords(text,
                                      keyphrase_ngram_range=ngram,
                                      stop_words='english',
                                      top_n=top_n,
                                      nr_candidates=nr_cand,
                                      use_mmr=True, diversity=diversity)
    return [k[0] for k in keyphrases]

# Initialize model
print("Loading KeyBERT model...")
model = KeyBERT()

Loading KeyBERT model...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Check the samples

In [9]:
# Process test samples
for i, sample in enumerate(test_samples):
    # Get original sentence and true entities
    sentence = sample['sentence']
    true_entities = re.findall(r'<e[12]>(.*?)</e[12]>', sentence)

    # Clean sentence and extract keyphrases
    clean_text = clean_sentence(sentence)
    # Preprocess
    preprocessed_text = preprocess_text(clean_text)
    ## Example for ngram (1,3) and nr_cand = 2 and diversity = 0.2
    extracted_phrases = extract_keyphrases((1,3), 2, 0.2, preprocessed_text, model)

    # Print results
    print(f"\nSample {i+1}:")
    print(f"Clean text: {clean_text}")
    print(f"True entities: {true_entities}")
    print(f"Extracted keyphrases: {extracted_phrases}")
    print("-" * 80)


Sample 1:
Clean text: The system as described above has its greatest application in an arrayed configuration of antenna elements.
True entities: ['configuration', 'elements']
Extracted keyphrases: ['arrayed configuration antenna', 'configuration antenna elements']
--------------------------------------------------------------------------------

Sample 2:
Clean text: The child was carefully wrapped and bound into the cradle by means of a cord.
True entities: ['child', 'cradle']
Extracted keyphrases: ['child carefully wrapped', 'cradle means cord']
--------------------------------------------------------------------------------

Sample 3:
Clean text: The author of a keygen uses a disassembler to look at the raw assembly code.
True entities: ['author', 'disassembler']
Extracted keyphrases: ['keygen uses disassembler', 'raw assembly code']
--------------------------------------------------------------------------------

Sample 4:
Clean text: A misty ridge uprises from the surge.
True enti

## Evaluation method

In [10]:
def evaluate_matches(true_entities: List[str], extracted_phrases: List[str], partial_match: bool = True) -> Tuple[float, float, float]:
    """Calculate precision, recall, and F1 score"""
    if partial_match:
        # Count each match only once by tracking which true entities have been matched
        matched_true = set()
        matched_extracted = set()

        for i, ext in enumerate(extracted_phrases):
            for j, gold in enumerate(true_entities):
                if (ext.lower() in gold.lower() or gold.lower() in ext.lower()):
                    matched_extracted.add(i)
                    matched_true.add(j)

        matches = len(matched_true)  # Count unique matches
    else:
        matches = sum(1 for ext in extracted_phrases
                     if any(ext.lower() == gold.lower() for gold in true_entities))

    precision = matches / len(extracted_phrases) if extracted_phrases else 0
    recall = matches / len(true_entities) if true_entities else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1



## Evaluate all samples

In [11]:
all_metrics = []
for i, sample in enumerate(test_samples):
    # Get entities and extract keyphrases
    true_entities = re.findall(r'<e[12]>(.*?)</e[12]>', sample['sentence'])
    clean_text = clean_sentence(sample['sentence'])
     # Preprocess
    preprocessed_text = preprocess_text(clean_text)
    ## Example for ngram (1,3) and nr_cand = 2 and diversity = 0.2
    extracted_phrases = extract_keyphrases((1,3), 2, 0.2, preprocessed_text, model)

    # Calculate metrics
    precision, recall, f1 = evaluate_matches(true_entities, extracted_phrases)
    all_metrics.append((precision, recall, f1))

    # Print results
    print(f"\nSample {i+1}:")
    print(f"True entities: {true_entities}")
    print(f"Extracted phrases: {extracted_phrases}")
    print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1: {f1:.2f}")

# Calculate average metrics
avg_precision = np.mean([m[0] for m in all_metrics])
avg_recall = np.mean([m[1] for m in all_metrics])
avg_f1 = np.mean([m[2] for m in all_metrics])

print("\nAverage Metrics:")
print(f"Precision: {avg_precision:.2f}")
print(f"Recall: {avg_recall:.2f}")
print(f"F1 Score: {avg_f1:.2f}")


Sample 1:
True entities: ['configuration', 'elements']
Extracted phrases: ['arrayed configuration antenna', 'configuration antenna elements']
Precision: 1.00, Recall: 1.00, F1: 1.00

Sample 2:
True entities: ['child', 'cradle']
Extracted phrases: ['child carefully wrapped', 'cradle means cord']
Precision: 1.00, Recall: 1.00, F1: 1.00

Sample 3:
True entities: ['author', 'disassembler']
Extracted phrases: ['keygen uses disassembler', 'raw assembly code']
Precision: 0.50, Recall: 0.50, F1: 0.50

Sample 4:
True entities: ['ridge', 'surge']
Extracted phrases: ['ridge uprises surge', 'misty ridge uprises']
Precision: 1.00, Recall: 1.00, F1: 1.00

Sample 5:
True entities: ['student', 'association']
Extracted phrases: ['university new york', 'undergraduate student population']
Precision: 0.50, Recall: 0.50, F1: 0.50

Sample 6:
True entities: ['complex', 'producer']
Extracted phrases: ['perus largest producer', 'largest producer silver']
Precision: 0.50, Recall: 0.50, F1: 0.50

Sample 7:
True

## Process dataset and save results

In [12]:
def process_dataset_in_batches(mode, ngram, nr_cand, diversity, dataset, model, batch_size=200, save_every=1000):
    """
    Process the dataset in batches
    """
    start_time = time.time()
    all_metrics = []
    all_results = []
    total_samples = len(dataset)

    # Create progress bar
    pbar = tqdm(total=total_samples, desc="Processing samples")

    for i in range(0, total_samples, batch_size):
        # Get batch
        batch = dataset.select(range(i, min(i + batch_size, total_samples)))
        batch_metrics = []
        batch_results = []

        # Process each sample in batch
        for sample in batch:
            true_entities = re.findall(r'<e[12]>(.*?)</e[12]>', sample['sentence'])
            clean_text = clean_sentence(sample['sentence'])# Preprocess
            preprocessed_text = preprocess_text(clean_text)
            ## Example for ngram (1,3) and nr_cand = 2
            extracted_phrases = extract_keyphrases(ngram, nr_cand, diversity, preprocessed_text, model)

            # Calculate metrics
            precision, recall, f1 = evaluate_matches(true_entities, extracted_phrases)
            batch_metrics.append((precision, recall, f1))

            # Store detailed results
            batch_results.append({
                # 'sentence': sample['sentence'],
                # 'true_entities': true_entities,
                # 'extracted_phrases': extracted_phrases,
                'precision': precision,
                'recall': recall,
                'f1'  : f1
                # 'metrics': {'precision': precision, 'recall': recall, 'f1': f1}
            })

        # Update main lists
        all_metrics.extend(batch_metrics)
        all_results.extend(batch_results)

        # Update progress bar
        pbar.update(len(batch))

    pbar.close()

    # Calculate final averages
    avg_metrics = calculate_average_metrics(all_metrics)

    # Print time taken
    time_taken = time.time() - start_time
    print(f"\nTotal time taken: {time_taken:.2f} seconds")

    save(mode, ngram, nr_cand, diversity, avg_metrics, time_taken)

    return all_results, avg_metrics

def save(mode, ngram, nr_cand, diversity, avg_metrics, time):
    """Save results to CSV file, appending new rows if the file exists."""
    # Define the filename
    filename = "/content/drive/MyDrive/NLP_Proj/Results/MMR_results.csv"

    # Prepare data for DataFrame
    data = {
               'mode' : mode,
                'ngram' : str(ngram),
                'nr_cand' : str(nr_cand),
                'diversity' : str(diversity),
                'avg_metrics' : avg_metrics,
                'time' : time
    }

    # Create a DataFrame
    df = pd.DataFrame(data)

    # Check if the file exists to determine mode
    if os.path.exists(filename):
        # Append to existing CSV
        df.to_csv(filename, mode='a', header=False, index=False)
    else:
        # Create a new CSV file with header
        df.to_csv(filename, mode='w', header=True, index=False)

    print(f"\nSaved results to {filename}")

def calculate_average_metrics(metrics):
    """Calculate average metrics from list of (precision, recall, f1) tuples"""
    avg_precision = np.mean([m[0] for m in metrics])
    avg_recall = np.mean([m[1] for m in metrics])
    avg_f1 = np.mean([m[2] for m in metrics])

    return {
        'precision': avg_precision,
        'recall': avg_recall,
        'f1': avg_f1
    }

# Process the full training dataset and test set

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ngrams = [(1,2), (1,3), (2,2), (2,3), (3,3)]
nr_cands = range(2,6)
diversity = [0.2, 0.3, 0.4, 0.5, 0.6]

print("Starting full dataset processing...")
for ngram in ngrams:
    for nr_cand in nr_cands:
      for div in diversity:
          print(f"\nProcessing ngram={ngram}, nr_cand={nr_cand}, diversity={div}")
        # Initialize model
          print("Loading KeyBERT model...")
          model = KeyBERT()
          results, avg_metrics = process_dataset_in_batches('Train', ngram, nr_cand, div, ds['train'], model)
          print("\nFinal Average Train Metrics:")
          print(f"Precision: {avg_metrics['precision']:.3f}")
          print(f"Recall: {avg_metrics['recall']:.3f}")
          print(f"F1 Score: {avg_metrics['f1']:.3f}")

          test_results, test_metrics = process_dataset_in_batches('Test', ngram, nr_cand, div, ds['test'], model)
          print("\nFinal Average Test Metrics:")
          print(f"Precision: {test_metrics['precision']:.3f}")
          print(f"Recall: {test_metrics['recall']:.3f}")
          print(f"F1 Score: {test_metrics['f1']:.3f}")





Starting full dataset processing...

Processing ngram=(1, 2), nr_cand=2, diversity=0.2
Loading KeyBERT model...


Processing samples:   0%|          | 0/8000 [00:00<?, ?it/s]


Total time taken: 188.93 seconds

Saved results to /content/drive/MyDrive/NLP_Proj/Results/MMR_results.csv

Final Average Train Metrics:
Precision: 0.599
Recall: 0.599
F1 Score: 0.599


Processing samples:   0%|          | 0/2717 [00:00<?, ?it/s]


Total time taken: 62.79 seconds

Saved results to /content/drive/MyDrive/NLP_Proj/Results/MMR_results.csv

Final Average Test Metrics:
Precision: 0.588
Recall: 0.588
F1 Score: 0.588

Processing ngram=(1, 2), nr_cand=2, diversity=0.3
Loading KeyBERT model...


...

Processing samples:   0%|          | 0/8000 [00:00<?, ?it/s]


Total time taken: 187.22 seconds

Saved results to /content/drive/MyDrive/NLP_Proj/Results/MMR_results.csv

Final Average Train Metrics:
Precision: 0.602
Recall: 0.602
F1 Score: 0.602


Processing samples:   0%|          | 0/2717 [00:00<?, ?it/s]


Total time taken: 64.74 seconds

Saved results to /content/drive/MyDrive/NLP_Proj/Results/MMR_results.csv

Final Average Test Metrics:
Precision: 0.596
Recall: 0.596
F1 Score: 0.596

Processing ngram=(1, 3), nr_cand=2, diversity=0.2
Loading KeyBERT model...


Processing samples:   0%|          | 0/8000 [00:00<?, ?it/s]

In [14]:
ngrams = [(1,3), (2,2), (2,3), (3,3)]
nr_cands = range(2,6)
diversity = [0.2, 0.3, 0.4, 0.5, 0.6]

print("Starting full dataset processing...")
for ngram in ngrams:
    for nr_cand in nr_cands:
      for div in diversity:
          print(f"\nProcessing ngram={ngram}, nr_cand={nr_cand}, diversity={div}")
        # Initialize model
          print("Loading KeyBERT model...")
          model = KeyBERT()
          results, avg_metrics = process_dataset_in_batches('Train', ngram, nr_cand, div, ds['train'], model)
          print("\nFinal Average Train Metrics:")
          print(f"Precision: {avg_metrics['precision']:.3f}")
          print(f"Recall: {avg_metrics['recall']:.3f}")
          print(f"F1 Score: {avg_metrics['f1']:.3f}")

          test_results, test_metrics = process_dataset_in_batches('Test', ngram, nr_cand, div, ds['test'], model)
          print("\nFinal Average Test Metrics:")
          print(f"Precision: {test_metrics['precision']:.3f}")
          print(f"Recall: {test_metrics['recall']:.3f}")
          print(f"F1 Score: {test_metrics['f1']:.3f}")


Starting full dataset processing...

Processing ngram=(1, 3), nr_cand=2, diversity=0.2
Loading KeyBERT model...


Processing samples:   0%|          | 0/8000 [00:00<?, ?it/s]


Total time taken: 239.37 seconds

Saved results to /content/drive/MyDrive/NLP_Proj/Results/MMR_results.csv

Final Average Train Metrics:
Precision: 0.719
Recall: 0.719
F1 Score: 0.719


Processing samples:   0%|          | 0/2717 [00:00<?, ?it/s]


Total time taken: 78.81 seconds

Saved results to /content/drive/MyDrive/NLP_Proj/Results/MMR_results.csv

Final Average Test Metrics:
Precision: 0.718
Recall: 0.718
F1 Score: 0.718

Processing ngram=(1, 3), nr_cand=2, diversity=0.3
Loading KeyBERT model...


...


Total time taken: 46.43 seconds

Saved results to /content/drive/MyDrive/NLP_Proj/Results/MMR_results.csv

Final Average Test Metrics:
Precision: 0.794
Recall: 0.764
F1 Score: 0.774

Processing ngram=(3, 3), nr_cand=5, diversity=0.6
Loading KeyBERT model...


Processing samples:   0%|          | 0/8000 [00:00<?, ?it/s]


Total time taken: 134.35 seconds

Saved results to /content/drive/MyDrive/NLP_Proj/Results/MMR_results.csv

Final Average Train Metrics:
Precision: 0.784
Recall: 0.758
F1 Score: 0.767


Processing samples:   0%|          | 0/2717 [00:00<?, ?it/s]


Total time taken: 45.55 seconds

Saved results to /content/drive/MyDrive/NLP_Proj/Results/MMR_results.csv

Final Average Test Metrics:
Precision: 0.782
Recall: 0.752
F1 Score: 0.762
